<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Embedding-Layer" data-toc-modified-id="Embedding-Layer-1">Embedding Layer</a></span></li><li><span><a href="#RNN" data-toc-modified-id="RNN-2">RNN</a></span></li><li><span><a href="#LSTM" data-toc-modified-id="LSTM-3">LSTM</a></span></li><li><span><a href="#GRU" data-toc-modified-id="GRU-4">GRU</a></span></li><li><span><a href="#Bidirectional-RNN" data-toc-modified-id="Bidirectional-RNN-5">Bidirectional RNN</a></span></li></ul></div>

# Embedding Layer

In [1]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy())    # 원-핫 인코딩 벡터를 출력해 봅시다.

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


In [2]:
distribution_size = 2   # 보기 좋게 2차원으로 분산 표현하도록 하죠!
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

print("Linear Weight")
print(linear.weights[0].numpy())

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight
[[-0.6576806   0.48297644]
 [ 0.547627   -0.56699896]
 [-0.49930015 -0.42260754]
 [-0.07189983  0.394207  ]
 [-0.22466624 -0.0750885 ]
 [-0.51240474  0.67449725]
 [ 0.2764722  -0.2711354 ]
 [-0.10633534  0.22316647]]

One-Hot Linear Result
[[-0.6576806   0.48297644]
 [-0.6576806   0.48297644]
 [-0.6576806   0.48297644]
 [-0.6576806   0.48297644]
 [ 0.547627   -0.56699896]
 [-0.49930015 -0.42260754]
 [-0.07189983  0.394207  ]
 [-0.22466624 -0.0750885 ]
 [-0.22466624 -0.0750885 ]
 [-0.22466624 -0.0750885 ]]


위처럼 원-핫 벡터에 Linear 레이어를 적용하면 Linear 레이어의 Weight에서 단어 인덱스 배열 \[ 0, 0, 0, 0, 1, 2, 3, 4, 4, 4 \] 에 해당하는 행만 읽어오는 효과가 있습니다. 많은 자연어처리 모델에서 문장 데이터 속 단어들을 단어 사전의 인덱스 숫자로 표현했다가 모델에 입력하는데, 사실은 이렇게 인덱스를 원-핫 임베딩으로 변환한 후 Embedding 레이어의 입력으로 넣어주는 처리를 하고 있는 것입니다. Tensorflow에서 Embedding 레이어를 선언하는 법은 아래와 같습니다.

In [3]:
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장입니다.

print("Embedding을 진행할 문장:", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언할 것이고,
# 각 단어는 100차원으로 분산표현 할 것입니다.

print("Embedding된 문장:", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

Embedding을 진행할 문장: (1, 3)
Embedding된 문장: (1, 3, 100)
Embedding Layer의 Weight 형태: (64, 100)


# RNN
- 시퀀스 데이터 처리
- (입력의 차원, 출력의 차원)에 해당하는 단 하나의 Weight를 순차적으로 업데이트
- 기울기 소실(Vanishing Gradient) 문제

In [4]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장:", sentence)

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위해 단어 매핑:", sentence_tensor.numpy())
print("입력 문장 데이터 형태:", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과:", emb_out.shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output):", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태:", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output):", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태:", rnn_fin_layer.weights[0].shape)

RNN에 입력할 문장: What time is it ?
Embedding을 위해 단어 매핑: [[2 3 0 1 4]]
입력 문장 데이터 형태: (1, 5)

Embedding 결과: (1, 5, 100)
Embedding Layer의 Weight 형태: (5, 100)

RNN 결과 (모든 Step Output): (1, 5, 64)
RNN Layer의 Weight 형태: (100, 64)

RNN 결과 (최종 Step Output): (1, 64)
RNN Layer의 Weight 형태: (100, 64)


# LSTM
- 기울기 소실 문제를 해결하기 위해 고안된 RNN 레이어
- Embedding 벡터의 차원수(unit)의 크기가 동일할 경우, SimpleRNN보다 4배 큰 Weight를 가짐(4종류의 서로 다른 Weight)
- 3개의 Gate Layer를 가짐
    - Forget Gate Layer : cell state의 기존 정보를 얼마나 잊어버릴지를 결정하는 gate
    - Input Gate Layer  : 새롭게 만들어진 cell state를 기존 cell state에 얼마나 반영할지를 결정하는 gate
    - Output Gate Layer : 새롭게 만들어진 cell state를 새로운 hidden state에 얼마나 반영할지를 결정하는 gate

In [5]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output):", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output): (1, 5, 64)
LSTM Layer의 Weight 형태: (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)


# GRU
- LSTM의 Forget Gate와 Inpurt Gate를 Update Gate로 합치고 Cell State와 Hidden State를 합쳐 비교적 간단한 구조를 가짐
- LSTM에 비해 학습할 가중치(Weight)가 1/4로 더 적음
- LSTM은 GRU에 비해 Weight가 많기 때문에 충분한 데이터가 있는 상황에 적합하고, 반대로 GRU는 적은 데이터에도 웬만한 학습 성능을 보여줌

# Bidirectional RNN
- 진행 방향이 반대인 RNN을 2개 겹쳐놓은 형태
- 기계번역에서 양방향(Bidirectional) RNN 계열의 네트워크, 혹은 동일한 효과를 내는 Transformer 네트워크를 주로 사용
- 순방향 Weight와 역방향 Weight를 각각 정의하므로 RNN의 2배 크기의 Weight를 가짐

In [6]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (최종 Step Output):", bi_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (최종 Step Output): (1, 5, 128)
